# NN training for Score

## 1. Posteriors and Scores
### a. Training Set

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  

k_nearest = 10
distance_offset =1e-5

with open('../input/siim-fisabio-rsna-classification-tl/train.npy', 'rb') as f:
    points_train = np.load(f)
    labels_train = np.load(f)

points = points_train
labels = labels_train
print('Shape of train points and labels',points.shape, labels.shape)

# Using the same variable names for ease and memory efficiency
X1 =np.zeros([len(labels)])
X2 =np.zeros([len(labels)])
y =np.zeros([len(labels)])

index_0 =0
index_1=0
ms_distance = np.zeros([len(labels_train),2])

for validation_index in range(len(labels)):
    point = points[validation_index,:]
    
    for iter1 in range(len(labels_train)):
        ms_distance[iter1,:] = [np.mean(np.power(point- points_train[iter1,:],2)), labels_train[iter1]]

    sorted_array = ms_distance[np.argsort(ms_distance[:, 0])]

    numerator = 0
    denominator = 0
    for iter1 in range(k_nearest):
        numerator = numerator + sorted_array[iter1,1]/(distance_offset + sorted_array[iter1,0]) # numerator increases only for label=1 
        denominator = denominator + 1/(distance_offset + sorted_array[iter1,0])

    opacity_probability = numerator/denominator
    
    X1[validation_index] = point[0]
    X2[validation_index] = point[1]
    y[validation_index] = opacity_probability
    #print([point[0], point[1], opacity_probability])  
    
    

dict = {'Input1': X1, 'Input2': X2, 'Output': y}  
df = pd.DataFrame(dict) 
df.to_csv('Data_Train.csv', header=True, index=False) 

Shape of train points and labels (5702, 2) (5702,)


### b. Validation Set

In [2]:
with open('../input/siim-fisabio-rsna-classification-tl/train.npy', 'rb') as f:
    points_train = np.load(f)
    labels_train = np.load(f)
print('Train points and labels',points.shape, labels.shape)

with open('../input/siim-fisabio-rsna-classification-tl/val.npy', 'rb') as f:
    points = np.load(f)
    labels = np.load(f)
print('Shape of validation points and labels',points.shape, labels.shape)

# Using the same variable names for ease and memory efficiency
X1v =np.zeros([len(labels)])
X2v =np.zeros([len(labels)])
yv =np.zeros([len(labels)])
index_0 =0
index_1=0
ms_distance = np.zeros([len(labels_train),2])

for validation_index in range(len(labels)):
    point = points[validation_index,:]
    
    for iter1 in range(len(labels_train)):
        ms_distance[iter1,:] = [np.mean(np.power(point- points_train[iter1,:],2)), labels_train[iter1]]

    sorted_array = ms_distance[np.argsort(ms_distance[:, 0])]

    numerator = 0
    denominator = 0
    for iter1 in range(k_nearest):
        numerator = numerator + sorted_array[iter1,1]/(distance_offset + sorted_array[iter1,0]) # numerator increases only for label=1 
        denominator = denominator + 1/(distance_offset + sorted_array[iter1,0])

    opacity_probability = numerator/denominator
    
    X1v[validation_index] = point[0]
    X2v[validation_index] = point[1]
    yv[validation_index] = opacity_probability
    #print([point[0], point[1], opacity_probability])  
    
    
dict = {'Input1': X1v, 'Input2': X2v, 'Output': yv}  
df = pd.DataFrame(dict) 
df.to_csv('Data_Val.csv', header=True, index=False) 

Train points and labels (5702, 2) (5702,)
Shape of validation points and labels (316, 2) (316,)


### c. Test Set

In [3]:
with open('../input/siim-fisabio-rsna-classification-tl/train.npy', 'rb') as f:
    points_train = np.load(f)
    labels_train = np.load(f)
print('Train points and labels',points.shape, labels.shape)

with open('../input/siim-fisabio-rsna-classification-tl/test.npy', 'rb') as f:
    points = np.load(f)
    labels = np.load(f)
print('Shape of validation points and labels',points.shape, labels.shape)

# Using the same variable names for ease and memory efficiency
X1t =np.zeros([len(labels)])
X2t =np.zeros([len(labels)])
yt =np.zeros([len(labels)])
index_0 =0
index_1=0
ms_distance = np.zeros([len(labels_train),2])

for validation_index in range(len(labels)):
    point = points[validation_index,:]
    
    for iter1 in range(len(labels_train)):
        ms_distance[iter1,:] = [np.mean(np.power(point- points_train[iter1,:],2)), labels_train[iter1]]

    sorted_array = ms_distance[np.argsort(ms_distance[:, 0])]

    numerator = 0
    denominator = 0
    for iter1 in range(k_nearest):
        numerator = numerator + sorted_array[iter1,1]/(distance_offset + sorted_array[iter1,0])
        denominator = denominator + 1/(distance_offset + sorted_array[iter1,0])

    opacity_probability = numerator/denominator
    
    X1t[validation_index] = point[0]
    X2t[validation_index] = point[1]
    yt[validation_index] = opacity_probability
    #print([point[0], point[1], opacity_probability])  
    
    
dict = {'Input1': X1t, 'Input2': X2t, 'Output': yt}  
df = pd.DataFrame(dict) 
df.to_csv('Data_Test.csv', header=True, index=False) 

Train points and labels (316, 2) (316,)
Shape of validation points and labels (316, 2) (316,)


## 2. Data Preparation

In [4]:
train_csv_path = './Data_Train.csv'
test_csv_path = './Data_Test.csv'
val_csv_path = './Data_Val.csv'

train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
train_df.head()

,Input1,Input2,Output
0,-0.606012,0.482524,0.660340
1,-1.005259,0.880387,1.000000
2,0.191841,-0.276844,0.304706
3,-0.922519,0.796420,0.945098
4,-0.951401,0.839575,0.791186


In [5]:
input_=[]
output_=[]
for row in train_df.iloc:
    input_.append((row[0:len(row)-1]).astype(float))
    output_.append(row[-1])
    
i_val=[]
o_val=[]
for row in val_df.iloc:
    i_val.append((row[0:len(row)-1]).astype(float))
    o_val.append(row[-1])
    
num_input = len(row)-1;
print(num_input)

####################################################
#
# This code is written with the help of the demo:
# https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379
#
####################################################
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import imageio


torch.manual_seed(1)    # reproducible

x = torch.tensor(input_).float()  
y = torch.tensor(output_).float()   
new_shape = (len(y), 1)
y = y.view(new_shape)


i_val = torch.tensor(i_val).float()  
o_val = torch.tensor(o_val).float()   
new_shape = (len(o_val), 1)
o_val = o_val.view(new_shape)



max_y = torch.max(y[:,0])
min_y =torch.min(y[:,0])

max_x = torch.max(x,dim=0)
min_x = torch.min(x,dim=0)

print(max_y, min_y, max_x.values, min_x.values)

range_y = max_y - min_y
range_x = max_x.values - min_x.values

print(range_x, range_y)

    #Normalizing
x = (x - min_x.values)/range_x
y = (y - min_y)/range_y

    #Normalizing
i_val = (i_val - min_x.values)/range_x
o_val = (o_val - min_y)/range_y

2
tensor(1.) tensor(0.) tensor([1.6996, 1.2725]) tensor([-1.3708, -1.7394])
tensor([3.0704, 3.0119]) tensor(1.)


## 3. NN Training

In [6]:
# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)
 
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x

# use the same net as before      
net = Net(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net)  # net architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


minimum_train_loss = 1e5
minimum_val_loss = 1e5
EPOCH = 2000

# start training
for epoch in range(EPOCH):
  
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net
    
    if epoch%200 == 199:
      prediction = net_opt(i_val)
      loss_val = loss_func(prediction, o_val)
      if loss_val<minimum_val_loss:
        minimum_val_loss = loss_val
        net_opt_val = net_opt
      print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))

Net(
  (hidden): Linear(in_features=2, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Epoch [200/2000], Loss: 0.0191, Minimum Loss 0.019132, Val Loss 0.016824  
Epoch [400/2000], Loss: 0.0190, Minimum Loss 0.019012, Val Loss 0.016766  
Epoch [600/2000], Loss: 0.0190, Minimum Loss 0.018953, Val Loss 0.016766  
Epoch [800/2000], Loss: 0.0189, Minimum Loss 0.018914, Val Loss 0.016750  
Epoch [1000/2000], Loss: 0.0190, Minimum Loss 0.018896, Val Loss 0.016750  
Epoch [1200/2000], Loss: 0.0189, Minimum Loss 0.018861, Val Loss 0.016750  
Epoch [1400/2000], Loss: 0.0189, Minimum Loss 0.018838, Val Loss 0.016750  
Epoch [1600/2000], Loss: 0.0189, Minimum Loss 0.018811, Val Loss 0.016729  
Epoch [1800/2000], Loss: 0.0189, Minimum Loss 0.018779, Val Loss 0.016729  
Epoch [2000/2000], Loss: 0.0190, Minimum Loss 0.018752, Val Loss 0.016729  


## 4. NN Testing

In [7]:
test_df = pd.read_csv(test_csv_path)

i_test=[]
o_test=[]
for row in val_df.iloc:
    i_test.append((row[0:len(row)-1]).astype(float))
    o_test.append(row[-1])


   
i_test, o_test = Variable(torch.tensor(i_test)).float(), Variable(torch.tensor(o_test).float())
new_shape = (len(o_test), 1)
o_test = o_test.view(new_shape)

    #Normalizing
i_test = (i_test - min_x.values)/range_x
o_test = (o_test - min_y)/range_y

prediction = net_opt_val(i_test)
loss_test = loss_func(prediction, o_test)

print("Normalized Test Loss",loss_test.detach().numpy())

loss_test = loss_test*range_y*range_y # As the loss function returns MSE

print("Test Loss",loss_test.detach().numpy())

Normalized Test Loss 0.01677799
Test Loss 0.01677799


## 5. Saving Model

In [8]:

PATH = "./best_model_S.pt"
torch.save(net_opt_val.state_dict(), PATH)


'''
#Guideline for loading model in future 

import sys
sys.modules[__name__].__dict__.clear()

device = torch.device('cpu')
net = Net(n_feature=num_input, n_hidden=500, n_output=1)

net.load_state_dict(torch.load(PATH, map_location=device))

'''

"\n#Guideline for loading model in future \n\nimport sys\nsys.modules[__name__].__dict__.clear()\n\ndevice = torch.device('cpu')\nnet = Net(n_feature=num_input, n_hidden=500, n_output=1)\n\nnet.load_state_dict(torch.load(PATH, map_location=device))\n\n"